In [ ]:
import numpy as np
import random  # for the random wirings
import time # to compare the algorithm timings

# I. Vectorialization

In [ ]:
"""CHSH = np.array([
    [0.25, 0, 0, 0.25],
    [0.25, 0, 0, 0.25],
    [0.25, 0, 0, 0.25],
    [0, 0.25, 0.25, 0]
])"""

CHSH = np.zeros((4,4))

for a in range(2):
    for b in range(2):
        for x in range(2):
            for y in range(2):
                if (a+b)%2 == x*y:
                    CHSH[2*x+y, 2*a+b]=0.25
                    
print(CHSH)

In [ ]:
def h(Q):
    return np.sum(Q * CHSH)  # produit scalair de Q et CHSH

In [ ]:
def h_flatten(Q):
    return np.dot( CHSH.flatten(), Q.flatten() )


In [ ]:
MatrixF = np.array([
    [-1, 0, 1, 0],
    [-1, 0, 1, 0],
    [0, -1, 0, 1],
    [0, -1, 0, 1],
    #
    [-1, 0, 1, 0],
    [-1, 0, 1, 0],
    [0, -1, 0, 1],
    [0, -1, 0, 1],
    #
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 1, 0, 0],
    #
    [1, 0, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 1, 0, 0]
])

MatrixG = np.array([
    [-1, 0, 1, 0],
    [0, -1, 0, 1],
    [-1, 0, 1, 0],
    [0, -1, 0, 1],
    #
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    #
    [-1, 0, 1, 0],
    [0, -1, 0, 1],
    [-1, 0, 1, 0],
    [0, -1, 0, 1],
    #
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

MatrixFtilda = np.array([
    [1, 0, 0, 0], 
    [1, 0, 0, 0], 
    [0, 1, 0, 0], 
    [0, 1, 0, 0],
    #
    [1, 0, 0, 0], 
    [1, 0, 0, 0], 
    [0, 1, 0, 0], 
    [0, 1, 0, 0],
    #
    [0, 0, 1, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 0, 0, 1],
    #
    [0, 0, 1, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 0, 0, 1]
])

MatrixGtilda = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0], 
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    #
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    #
    [1, 0, 0, 0],
    [0, 1, 0, 0], 
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    #
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

In [ ]:
def vector_to_matrix(vector):   # turns a vector of length 16 into a 4x4 matrix, columnwise
    return np.column_stack((vector[0:4], vector[4:8], vector[8:12], vector[12:16]))


def BoxProduct(W, P, Q):
    new_box = np.zeros((4,4))   # null 4x4 matrix
    
    for k in range(16):
        y = k%2
        x = ((k-y)//2)%2
        b = ((k-2*x-y)//4)%2
        a = ((k-4*b-2*x-y)//8)%2
        
        # f1(x, 0) = W[2*x]
        # f1(x, 1) = W[2*x+1]
        # g1(y, 0) = W[2*y+4]
        # g1(y, 1) = W[2*y+5]
        term1 = vector_to_matrix( np.dot(MatrixF, np.array([W[2*x], W[2*x+1], 1, 1])) *  np.dot(MatrixG, np.array([W[2*y+4], W[2*y+5], 1, 1])) )
        
        # f2(x, 0) = W[2*x+8]
        # f2(x, 1) = W[2*x+9]
        # g2(y, 0) = W[2*y+12]
        # g2(y, 1) = W[2*y+13]
        term2 = vector_to_matrix( np.dot(MatrixF, np.array([W[2*x+8], W[2*x+9], 1, 1])) * np.dot(MatrixG, np.array([W[2*y+12], W[2*y+13], 1, 1]))  )
        
        # The vector with the f3's is W[16:20] or W[20:24] depending on x
        term3 = (1-a)*np.ones((4,4)) - (-1)**a * vector_to_matrix( np.dot(MatrixFtilda, np.array(W[16+4*x: 20+4*x]))  )
        
        # The vector with the g3's is W[24:28] or W[28:32] depending on y
        term4 = (1-b)*np.ones((4,4)) - (-1)**b * vector_to_matrix( np.dot(MatrixGtilda, np.array(W[24+4*y: 28+4*y]))  )

        
        new_box[2*x+y, 2*a+b] = np.sum( np.dot(term1, P) * np.dot(term2, Q).T * term3 * term4 )
        
    return new_box



In [ ]:
# Fixer des boites P et Q

def phi(W, P, Q):
    return - h(BoxProduct(W, P, Q)) 

# II. Boxes

In [ ]:
"""PR = np.array([                     
    [0.5, 0, 0, 0.5],
    [0.5, 0, 0, 0.5],
    [0.5, 0, 0, 0.5],
    [0, 0.5, 0.5, 0]
])
    
          
SR = np.array([
    [0.5, 0, 0, 0.5],
    [0.5, 0, 0, 0.5],
    [0.5, 0, 0, 0.5],
    [0.5, 0, 0, 0.5]
])"""

PR = np.zeros((4,4))

for a in range(2):
    for b in range(2):
        for x in range(2):
            for y in range(2):
                if (a+b)%2 == x*y:
                    PR[2*x+y, 2*a+b]=0.5
                    
print("PR=\n",PR)

SR = np.zeros((4,4))

for a in range(2):
    for b in range(2):
        for x in range(2):
            for y in range(2):
                if a==b:
                    SR[2*x+y, 2*a+b]=0.5
                    
print("\nSR=\n",SR)

In [ ]:
import timeit

In [ ]:
start_time = time.time()
print(h(PR))

print("")
print("Single evaluation")
print("Duration: --- %s ms ---" % (1e3*(time.time() - start_time)) )

count=int(1e4)
print("")
print("Multiple evaluations of h: ", count)
counter = timeit.timeit(lambda: h(PR), number=count)
print( "Average per evaluation: %s ms" % (1e3*counter/count) )

count=int(1e4)
print("")
print("Multiple evaluations of phi: ", count)
counter = timeit.timeit(lambda: phi(W_BS09, PR, PR), number=count)
print( "Average per evaluation: %s ms" % (1e3*counter/count) )


In [ ]:
def P_L(mu, nu, sigma, tau):
    new_box = np.zeros((4,4))
    
    for a in range(2):
        for b in range(2):
            for x in range(2):
                for y in range(2):
                    if a==(mu*x+nu)%2 and b==(sigma*y+tau)%2:
                        new_box[2*x+y, 2*a+b] = 1
                        
    return new_box
                        

In [ ]:
P_0 = P_L(0,0,0,0)
P_1 = P_L(0,1,0,1)

In [ ]:
def P_NL(mu, nu, sigma):
    new_box = np.zeros((4,4))
    
    for a in range(2):
        for b in range(2):
            for x in range(2):
                for y in range(2):
                    if (a+b)%2==(x*y + mu*x + nu*y + sigma)%2:
                        new_box[2*x+y, 2*a+b] = 0.5
                        
    return new_box

In [ ]:
P_NL(0,0,0) == PR

In [ ]:
PRbar = P_NL(0,0,1)
print(PRbar/2 + PR/2)

# III. Wirings

In [ ]:
central_wiring = [0.5 for _ in range(32)]

def random_wiring():
    return [random.random() for _ in range(32)]

In [ ]:
W_BS09 = [0, 0, 1, 1,              # f_1(x, a_2) = x
          0, 0, 1, 1,              # g_1(y, b_2) = y
          0, 0, 0, 1,              # f_2(x, a_1) = a_1*x
          0, 0, 0, 1,              # g_2(y, b_1) = b_1*y
          0, 1, 1, 0, 0, 1, 1, 0,  # f_3(x, a_1, a_2) = a_1 + a_2 mod 2
          0, 1, 1, 0, 0, 1, 1, 0   # g_3(y, b_1, b_2) = b_1 + b_2 mod 2
         ]

W_ABLPSV09 = [
          0, 0, 1, 1,              # f_1(x, a_2) = x
          0, 0, 1, 1,              # g_1(y, b_2) = y
          1, 0, 0, 1,              # f_2(x, a_1) = x ⊕ a_1 ⊕ 1
          0, 0, 0, 1,              # g_2(y, b_1) = y*b_1
          1, 0, 0, 1, 1, 0, 0, 1,  # f_3(x, a_1, a_2) = a_1 ⊕ a_2 ⊕ 1
          1, 0, 0, 1, 1, 0, 0, 1,  # g_3(y, b_1, b_2) = b_1 ⊕ b_2 ⊕ 1
]


def wiring_to_functions(W):
    print("f_1(x,a2) = ", (W[2]-W[0])%2, "x ⊕ ", (W[1]-W[0])%2 ,"a2 ⊕ ", (W[3]-W[2]-W[1]+W[0])%2 ,"x*a2 ⊕ ", (W[0])%2)
    print("g_1(y,b2) = ", (W[6]-W[4])%2, "y ⊕ ", (W[5]-W[4])%2 ,"b2 ⊕ ", (W[7]-W[6]-W[5]+W[4])%2 ,"y*b2 ⊕ ", (W[4])%2)
    print("f_2(x,a1) = ", (W[10]-W[8])%2, "x ⊕ ", (W[9]-W[8])%2 ,"a1 ⊕ ", (W[11]-W[10]-W[9]+W[8])%2 ,"x*a1 ⊕ ", (W[8])%2)
    print("g_2(y,b1) = ", (W[14]-W[12])%2, "y ⊕ ", (W[13]-W[12])%2 ,"b1 ⊕ ", (W[15]-W[14]-W[13]+W[12])%2 ,"y*b1 ⊕ ", (W[12])%2)
    print("f_3(x,a1,a2) = ", (W[20]-W[16])%2, "x ⊕ ", (W[18]-W[16])%2 ,"a1 ⊕ ", (W[17]-W[16])%2 ,"a2 ⊕ ", 
          (W[21]-W[20]-W[18]+W[16])%2 ,"x*a1 ⊕ ", (W[22]-W[20]-W[17]+W[16])%2 ,"x*a2 ⊕ ", 
          (W[19]-W[18]-W[17]+W[16])%2 ,"a1*a2 ⊕ ", 
          (W[23] - W[21] - W[22]+W[20] - W[19]+W[18]+W[17] - W[16])%2,"x*a1*a2 ⊕ ", (W[16])%2)
    print("g_3(y,b1,b2) = ", (W[28]-W[24])%2, "y ⊕ ", (W[26]-W[24])%2 ,"b1 ⊕ ", (W[25]-W[24])%2 ,"b2 ⊕ ", 
          (W[29]-W[28]-W[26]+W[24])%2 ,"y*b1 ⊕ ", (W[30]-W[28]-W[25]+W[24])%2 ,"y*b2 ⊕ ", 
          (W[27]-W[26]-W[25]+W[24])%2 ,"b1*b2 ⊕ ", 
          (W[31] - W[29] - W[30]+W[28] - W[27]+W[26]+W[25] - W[24])%2,"y*b1*b2 ⊕ ", (W[16])%2)

### Power of a box

In [ ]:
def power_box(P, n, W):
    # Initialization
    term1 = np.zeros((4,4, 4,4))
    constant_term = np.zeros((4,4, 4,4))
    for k in range(16):
        y = k%2
        x = ((k-y)//2)%2
        b = ((k-2*x-y)//4)%2
        a = ((k-4*b-2*x-y)//8)%2
        
        term1[2*x+y, 2*a+b] = vector_to_matrix( np.dot(MatrixF, np.array([W[2*x], W[2*x+1], 1, 1])) *  np.dot(MatrixG, np.array([W[2*y+4], W[2*y+5], 1, 1])) )
        
        term2 = vector_to_matrix( np.dot(MatrixF, np.array([W[2*x+8], W[2*x+9], 1, 1])) * np.dot(MatrixG, np.array([W[2*y+12], W[2*y+13], 1, 1]))  )
        term3 = (1-a)*np.ones((4,4)) - (-1)**a * vector_to_matrix( np.dot(MatrixFtilda, np.array(W[16+4*x: 20+4*x])) )
        term4 = (1-b)*np.ones((4,4)) - (-1)**b * vector_to_matrix( np.dot(MatrixGtilda, np.array(W[24+4*y: 28+4*y])) )
        
        constant_term[2*x+y, 2*a+b] = np.dot(term2, P).T * term3 * term4                                                       
                                                                  
    # Induction: we compute the n-th power
    new_box = P.copy()
    for i in range(n-1):
        aux = new_box.copy()
        for k in range(16):
            y = k%2
            x = ((k-y)//2)%2
            b = ((k-2*x-y)//4)%2
            a = ((k-4*b-2*x-y)//8)%2                                                         
                                                                  
            #new_box[2*x+y, 2*a+b] = np.sum( np.dot(term1[2*x+y, 2*a+b], new_box) * (constant_term[2*x+y, 2*a+b]) )
            new_box[2*x+y, 2*a+b] = np.sum( np.dot(term1[2*x+y, 2*a+b], aux) * (constant_term[2*x+y, 2*a+b]) )
        
        
    return new_box

def power_box_left(P, n, W):
    # Initialization
    term2 = np.zeros((4,4, 4,4))
    constant_term = np.zeros((4,4, 4,4))
    for k in range(16):
        y = k%2
        x = ((k-y)//2)%2
        b = ((k-2*x-y)//4)%2
        a = ((k-4*b-2*x-y)//8)%2
        
        term2[2*x+y, 2*a+b] = vector_to_matrix( np.dot(MatrixF, np.array([W[2*x+8], W[2*x+9], 1, 1])) *  np.dot(MatrixG, np.array([W[2*y+12], W[2*y+13], 1, 1])) )
        
        term1 = vector_to_matrix( np.dot(MatrixF, np.array([W[2*x], W[2*x+1], 1, 1])) * np.dot(MatrixG, np.array([W[2*y+4], W[2*y+5], 1, 1]))  )
        term3 = (1-a)*np.ones((4,4)) - (-1)**a * vector_to_matrix( np.dot(MatrixFtilda, np.array(W[16+4*x: 20+4*x])) )
        term4 = (1-b)*np.ones((4,4)) - (-1)**b * vector_to_matrix( np.dot(MatrixGtilda, np.array(W[24+4*y: 28+4*y])) )
        
        constant_term[2*x+y, 2*a+b] = np.dot(term1, P) * term3 * term4                                                       
                                                                  
    # Induction: we compute the n-th power
    new_box = P.copy()
    for i in range(n-1):
        aux = new_box.copy()
        for k in range(16):
            y = k%2
            x = ((k-y)//2)%2
            b = ((k-2*x-y)//4)%2
            a = ((k-4*b-2*x-y)//8)%2                                                         
                                                                  
            new_box[2*x+y, 2*a+b] = np.sum( np.dot(term2[2*x+y, 2*a+b], aux).T * (constant_term[2*x+y, 2*a+b]) )
        
        
    return new_box

def power_box_BS09(P, n, W):
    new_box = P.copy()
    for i in range(n-1):
        new_box = BoxProduct(W, new_box, new_box)
    return new_box

n=10

def phi_n(W):
    return - h(power_box(corNLB(0.5), n, W))